<a href="https://colab.research.google.com/github/slz4025/direct_democracy_toy/blob/master/Direct_Democracy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Direct Democracy

View this file by putting its link [https://github.com/slz4025/direct_democracy_toy/blob/master/Direct_Democracy.ipynb] here: http://colab.research.google.com/github.

## Introduction

Direct democracies allow each member of a society to voice their views to the whole of society.  However, organizing people effectively to have productive discussions becomes more difficult with more people.  People tend to work best if the groups are smaller than ten people (Agile).  

Despite the downfalls of representative democracies in diminishing the voice of those who are non-representatives, it does offer a general structure that collects views and makes decisions--that of the tree structure (hierarchical structure).  

## Proposed System

Therefore, let us consider the following formal system for implementing direct democracy that we will call technological direct democracy.  With `N` people, let us divide them into smaller and smaller groups in a tree structure with a fixed branching constant such that the leaf nodes have `n` people, where `n <= 10`.  Now, discussion among the people will take place for `T` units of time and some number of rounds, each of length `t`.  Within each round, the participants of each leaf node discuss the particular issue at hand, and then summarize their discussions. In a representative democracy, discussion would continue at the parent nodes.  However, here, we try to maximize discussion at the leaf nodes, and thus minimize discussion at the parent nodes.  Though the summaries are passed up the tree to the parent node, all the summaries that arrive at a parent node are simply further summarized, then passed up again.  At the root node, the global summary should contain the general sense of the discussion that took place at the lower levels and no more. This global summary is then passed to each leaf group, so that its participants can continue to discuss the issue but now with a better knowledge of the global discussion.  In this way, we can use small, feasible discussion sizes, but also allow people to be aware of a large landscape of experience.

Such a system should ideally be used for complex issues, namely issues that cannot simply be broken into sub-parts that can then be discussed by a further division of people.  


## Proposed Problem and Model

Let us consider a class of problems that could be addressed by "direct action", Agile, the iterative design process, etc.  We introduce the following model.

TODO

Setup, including imports, hyperparameters specifying characteristics of the problems, individuals, democracies.

In [0]:
# imports
from random import shuffle, choice, randint
import numpy as np
import scipy.stats as st
from matplotlib import pyplot as plt
import math

In [0]:
# hyperparameters for the problem class



In [0]:
# hyperparameters for the individual model



In [0]:
# hyperparameters for the technological direct democracy

# hard constraints
HARD_MAX_GROUP_SIZE = 10                                                        # agile group size
HARD_MIN_GROUP_SIZE = 2                                                         # an exchange of ideas
HARD_MIN_BRANCH_FACTOR = 2                                                      # minimum possible branching factor

# structure constants
TOTAL_TIME = 1000                                                               # total length of discussion
ROUND_TIME = 10                                                                 # length of discussion round
ROUND_NUM = TOTAL_TIME // ROUND_TIME                                            # number of discussion rounds
TOTAL_IND = 100                                                                # total number of people
GROUP_SIZE = 6                                                                  # size of discussion group
MIN_GROUP_SIZE = 3                                                              # used min group size
BRANCH_FACTOR = 3                                                               # branching constant
MIN_BRANCH_FACTOR = 2                                                           # minimum branching constant

# check constants are valid
assert(GROUP_SIZE <= HARD_MAX_GROUP_SIZE)
assert(MIN_GROUP_SIZE >= HARD_MIN_GROUP_SIZE)
assert(MIN_GROUP_SIZE <= GROUP_SIZE)
assert(MIN_BRANCH_FACTOR <= HARD_MIN_BRANCH_FACTOR)

A small library of basic helper functions.

In [0]:
# helper functions

def show_array(A):
  data = np.uint8(A * 255)
  #https://stackoverflow.com/questions/33828780/matplotlib-display-array-values-with-imshow
  fig = plt.figure(figsize=(4, 3))
  ax = fig.add_subplot(111)
  im = ax.imshow(data, origin='lower', \
    interpolation='None', cmap='gray', vmin=0,vmax=255)
  plt.show()

def flatten(L): # one-level, can make recursive
  S = [e for l in L for e in l]
  return S

Here, we model an instance of a the class of problems that are design-like in nature and could be "solved" with "direct action", Agile, iterative design.

Here, we describe our model of the individual.

In [0]:
# class to govern how an individual works
class Individual:

  def __init__(self):
 
  def speak(self, discussion):


The following is the abstraction for a group, which is the basic unit of society.  It could be said to be the leaf node of the representative and technological direct democracies, and a small instance of a traditional direct democracy.

In [0]:
class Group:
  def __init__(self, members):
    self.members = members
    self.speaker = randint(0, len(self.members) - 1)

  def discuss(self, summary):
    contribution = self.members[self.speaker].speak(summary)
    # DECAY param
    summary = summary * 0.9 + contribution * 0.1
    self.speaker = (self.speaker + 1) % len(self.members)
    return summary

In [0]:
# main

individuals = []
for i in range(GROUP_SIZE):
  individual = Individual(simple_preference)
  #print("Individual {}\n".format(i))
  #show_array(individual.preference)
  individuals.append(individual)

unit = Group(individuals)

# TODO init summary

In [0]:
for i in range(TOTAL_TIME):
  summary = unit.discuss(summary)

The following is the abstraction for the technological direct democracy model.

NOTES:
* It would be nice/useful to have a visualization of the tree/member structure.

In [0]:
class LeafGroup:
  def __init__(self, members, parent):
    self.parent = parent
    self.members = members

  def discuss(self, global_summary):
    # TODO ################# make this generic
    speaker = randint(0, len(self.members) - 1)
    for i in range(ROUND_TIME):
      discussion = self.members[speaker].speak(discussion, contribution_weight)
      speaker = (speaker + 1) % len(self.members)
    discussion_summary = discussion 
    ########################
    return discussion_summary

  def summarize(self, global_summary):
    discussion_summary = self.discuss(global_summary)
    return discussion_summary

In [0]:
class ParentGroup:
  def __init__(self, members, parent, summarize_func):
    self.parent = parent
    self.children = []
    self.summarize_func = summarize_func
    
    assert(len(members) > GROUP_SIZE)
    assert(len(members) >= MIN_BRANCH_FACTOR * MIN_GROUP_SIZE)
    branch_factor = BRANCH_FACTOR
    children_size = len(members) // branch_factor
    if children_size > GROUP_SIZE: # children are parents
      while (len(members) > 2 * children_size):
        group = members[:children_size]
        members = members[children_size:]
        self.children.append(ParentGroup(group, self, summarize_func))
      assert(len(members) >= children_size)
      self.children.append(ParentGroup(members, self, summarize_func))
    else: # children are leaves
      if children_size <= MIN_GROUP_SIZE:
        branch_factor = len(members) // MIN_GROUP_SIZE
        children_size = len(members) // branch_factor # may != MIN_GROUP_SIZE
        # consider 16 // 6 = 2, 16 // 2 = 8
      while (len(members) >= 2 * children_size):
        group = members[:children_size]
        members = members[children_size:]
        self.children.append(LeafGroup(group, self))
      assert(len(members) >= children_size)
      self.children.append(LeafGroup(members, self))
  
  def summarize(self, global_summary):
    summaries = [c.summarize(global_summary) \
      for c in self.children]
    summary = self.summarize_func(summaries)

    return summary

In [0]:
# class for the proposed direct democracy structure
class DirectDemocracy:

  def __init__(self, summary_func, members, round_num,):
    shuffle(members) # random
    self.root_group = ParentGroup(members, None, summary_func)

  def run_round(self):
    summary = self.root_group.summarize(self.global_summary)
    return summary # change

In [0]:
# main

# create individuals with preferences
individuals = []
for i in range(TOTAL_IND):
  individual = Individual(simple_preference)
  #print("Individual {}\n".format(i))
  #show_array(individual.preference)
  individuals.append(individual)

# organize individuals into groups
dd = DirectDemocracy(summarize_average, individuals, ROUND_NUM)

# TODO init summary

In [0]:
# run each round
for i in range(ROUND_NUM):
  global_summary = dd.run_round()